In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
## input training set ## 90960 x 20 df
trainDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\df_train24_std.csv")
#trainDEFSDf[trainDEFSDf.type .== 1, :]
## calculate weight ## 0: 47449, 1: 43511
Yy_train = trainDEFSDf["type"]  # 0.9585; 1.0453
sampleW = []
for w in Yy_train:
    if w == 0:
        sampleW.append(0.9585)
    elif w == 1:
        sampleW.append(1.0453) 


In [4]:
## input ext val set ## 6075 x 20 df
extDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\df_ext24_std.csv")
#extDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 2943, 1: 3132
Yy_ext = extDEFSDf["type"]  # 1.0321; 0.9698
sampleExtW = []
for w in Yy_ext:
    if w == 0:
        sampleExtW.append(1.0321)
    elif w == 1:
        sampleExtW.append(0.9698) 


In [5]:
## reconstruct a whole set ## 97035 x 20 df
ingestedDEFSDf = pd.concat([trainDEFSDf, extDEFSDf])
#ingestedDEFSDf[ingestedDEFSDf.type .== 1, :]
## calculate weight ## 0: 50392, 1: 46643
Yy_ingested = ingestedDEFSDf["type"]  # 0.9628; 1.0402
sampleIngestedW = []
for w in Yy_ingested:
    if w == 0:
        sampleIngestedW.append(0.9628)
    elif w == 1:
        sampleIngestedW.append(1.0402)


In [6]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\df_FNA24_std.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 44540, 1: 44161
Yy_FNA = fnaDEFSDf["type"]  # 0.9957; 1.0043
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.9957)
    elif w == 1:
        sampleFNAW.append(1.0043)


In [7]:
fnaDEFSDf

,pixel_id,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,633.5108,445.3172,683.5948,269.0878,215.0328,283.255,671.4661,610.5128,175.0246,type
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,-0.005702,0.008693,-0.163091,0.119808,-0.105292,-0.057695,-0.154677,-0.247699,-0.072969,-0.063891,-0.02283,-0.139713,0.615396,0.249740,-0.306879,-0.043562,-0.020918,0.270866,0
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,-0.005702,-0.225150,0.069084,0.422357,-0.105292,-0.057695,-0.154677,-0.247699,-0.072969,-0.063891,-0.02283,-0.139713,0.686542,0.241333,-0.306879,-0.067193,-0.020918,0.341769,0
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,-0.005702,-0.225150,0.101513,0.374833,-0.105292,-0.057695,-0.154677,-0.247699,-0.072969,-0.063891,-0.02283,-0.139713,0.767749,0.247104,-0.306879,-0.029138,-0.020918,0.366825,0
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,-0.005702,-0.225150,0.663425,-0.143943,-0.105292,-0.057695,-0.154677,-0.247699,0.788622,-0.063891,-0.02283,-0.139713,0.718588,0.445671,-0.306879,-0.402832,-0.020918,0.439434,0
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,-0.005702,-0.225150,0.603374,-0.143943,-0.105292,-0.057695,-0.154677,-0.247699,0.630092,-0.063891,-0.02283,-0.139713,0.698350,0.522524,-0.306879,-0.402832,-0.020918,0.521852,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88696,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_6...,-0.005702,-0.225150,0.321189,0.163895,-0.105292,-0.057695,-0.154677,-0.247699,-0.072969,-0.063891,-0.02283,-0.139713,-0.102398,-0.225931,0.011566,-0.402832,-0.020918,-0.167027,1
88697,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_6...,-0.005702,-0.225150,0.322064,-0.143943,-0.105292,-0.057695,-0.154677,-0.247699,-0.072969,-0.063891,-0.02283,-0.139713,-0.102398,-0.225931,0.137602,-0.402832,-0.020918,-0.167027,1
88698,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_6...,-0.005702,-0.225150,0.364526,0.153275,-0.105292,-0.057695,-0.154677,-0.247699,-0.072969,-0.063891,-0.02283,-0.139713,-0.102398,-0.225931,0.054128,-0.402832,-0.020918,-0.167027,1
88699,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_6...,-0.005702,-0.225150,0.389958,-0.143943,-0.105292,-0.057695,-0.154677,-0.247699,0.294053,-0.063891,-0.02283,-0.139713,-0.102398,-0.225931,-0.306879,-0.161811,-0.020918,-0.167027,1


In [8]:
## define functions for performace evaluation ##
    # Maximum absolute error
# mean square error (MSE) calculation
# Root mean square error (RMSE) calculation
def errorDetermination(arrRi, predictedRi):
    sumAE = 0
    maxAE = 0
    for i in range(len(predictedRi)):
        AE = abs(arrRi[i] - predictedRi[i])
        if (AE > maxAE):
            maxAE = AE
        sumAE += (AE ** 2)
        
    MSE = sumAE / len(predictedRi)
    RMSE = MSE ** 0.5
    return maxAE, MSE, RMSE

#
# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [9]:
trainDEFSDf[trainDEFSDf.columns[1:-1]]

,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,633.5108,445.3172,683.5948,269.0878,215.0328,283.255,671.4661,610.5128,175.0246
0,-0.256907,-0.000982,0.142569,0.017134,0.126088,0.080817,-0.058970,-0.073677,-0.187081,-0.281360,-0.095382,-0.181833,-0.030762,-0.119108,-0.024738,0.150376,-0.167473,-0.004607
1,0.158221,-0.010497,0.374051,-0.011979,0.027052,-0.328894,0.024663,-0.044434,-0.187081,0.032998,0.222220,-0.181833,0.150816,-0.038276,-0.006922,0.141125,-0.167473,0.083503
2,-0.256907,-0.210743,0.157616,0.042145,0.017886,0.191653,0.015601,0.019519,-0.187081,-0.281360,-0.095382,0.139729,0.023640,-0.083684,0.023263,0.203888,-0.167473,-0.237962
3,-0.256907,-0.022928,0.109249,0.032018,0.120608,-0.328894,0.008461,0.058171,-0.187081,-0.281360,-0.095382,0.078984,-0.070448,-0.042441,0.023771,0.278755,-0.167473,-0.004370
4,-0.256907,-0.210743,0.189790,-0.175445,0.072955,-0.328894,-0.023746,-0.223288,-0.187081,-0.281360,-0.095382,-0.181833,0.065576,-0.131907,-0.096229,0.208789,0.192620,0.015235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,-0.256907,-0.210743,-0.257989,-0.175445,-0.272238,-0.328894,-0.339638,-0.223288,-0.187081,-0.281360,-0.095382,-0.181833,0.104503,0.004160,-0.014514,-0.211163,-0.167473,0.099762
90956,-0.256907,-0.210743,-0.257989,0.043667,-0.272238,-0.328894,-0.339638,-0.223288,-0.187081,-0.281360,-0.095382,-0.181833,0.096594,-0.091239,0.155709,-0.211163,-0.167473,-0.237962
90957,-0.256907,-0.210743,0.315221,0.059179,-0.272238,-0.328894,0.150399,-0.027649,0.225073,-0.281360,0.456950,-0.181833,0.042333,-0.093394,0.122536,-0.211163,-0.167473,-0.237962
90958,-0.256907,-0.210743,-0.257989,0.050129,-0.272238,-0.328894,-0.339638,-0.223288,0.129701,-0.281360,0.353228,-0.181833,0.095210,-0.122676,0.039255,-0.211163,0.412686,-0.237962


In [25]:
## define a function for Gradient Boost ##
def optimXGBoostClass(inputDB, inputDB_ingested, inputDB_ext, inputDB_FNA):
    #lr_r = [0.3, 0.2, 0.1, 0.05]  # 4
    #lr_r = [5, 2.5, 1, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01, 0.005, 0.0025, 0.001]  # 4
    #lr_r = [10, 7.5, 5, 2.5, 1, 0.75, 0.5, 0.4, 0.3, 0.2, 0.1, 0.075, 0.05]  # 13
    lr_r = [100, 75, 50, 45, 40, 35, 30, 25, 20, 15, 10]  # 11
    #depth_r = [4, 6, 8, 10]  # 4
    #depth_r = [2, 4, 5, 6, 7, 8, 9, 10, 11, 12]  # 4
    depth_r = [5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18]  # 11
    #tree_r = [50, 75, 100, 125]  # 4
    tree_r = [25, 50, 100]  # 3
    #est_r = [1, 3, 5, 10, 15] # 5
    #est_r = [2, 5, 10] # 3
    est_r = [2, 3, 5, 7, 10] # 5

    rs = 42
    z = np.zeros(49)
    itr = 1

    N_train = inputDB
    M_train = inputDB_ingested
    M_ext = inputDB_ext
    M_FNA = inputDB_FNA

    for lr in lr_r:
        for d in depth_r:
            for t in tree_r:
                for est in est_r:
    
                    print("itr=", itr, ", lr=", lr, ", est=", est, ", depth=", d, ", tree=", t)
                    print("## loading in data ##")
                    Xx_train = M_train[M_train.columns[1:-1]]
                    nn_train = N_train[N_train.columns[1:-1]]
                    Xx_Ext = M_ext[M_ext.columns[1:-1]]
                    Xx_FNA = M_FNA[M_FNA.columns[1:-1]]
                    #
                    Yy_train = M_train["type"]
                    mm_train = N_train["type"]
                    Yy_Ext = M_ext["type"]
                    Yy_FNA = M_FNA["type"]
                    print("## Classification ##")
                    model_xgb = xgb.XGBClassifier(
                        base_score = 0.5, 
                        booster = 'gbtree', 
                        colsample_bylevel = 1, 
                        colsample_bynode = 1, 
                        colsample_bytree = 1, 
                        enable_categorical = False, 
                        eval_metric = 'auc', 
                        gamma = 0, 
                        gpu_id = -1, 
                        grow_policy = 'lossguide', 
                        importance_type = None, 
                        learning_rate = lr, 
                        max_bin = 256, 
                        max_cat_to_onehot = 4, 
                        max_delta_step = 0, 
                        max_depth = d, 
                        max_leaves = 0, 
                        min_child_weight = 1, 
                        #missing = nan, 
                        monotone_constraints = '()', 
                        n_estimators = t, 
                        n_jobs = 0, 
                        num_parallel_tree = 1, 
                        objective = 'binary:hinge', 
                        predictor = 'auto', 
                        random_state = 0, 
                        reg_alpha = 0, 
                        reg_lambda = 1, 
                        sampling_method = 'uniform', 
                        scale_pos_weight = None, 
                        subsample = 1, 
                        tree_method = 'hist', 
                        use_label_encoder = False, 
                        validate_parameters = 1, 
                        verbosity = None, 
                        early_stopping_rounds = est)

                    # Training Set
                    model_xgb.fit(Xx_train, Yy_train, 
                                  eval_set=[(Xx_train, Yy_train), (Xx_FNA, Yy_FNA)])

                    importances = permutation_importance(model_xgb, Xx_FNA, Yy_FNA, n_repeats=10, random_state=42, n_jobs=-1)
                    print(importances["importances_mean"])
                    
                    if itr == 1:
                        z[0] = lr
                        z[1] = est
                        z[2] = d
                        z[3] = t
                        z[4] = f1_score(mm_train, model_xgb.predict(nn_train), sample_weight=sampleW)
                        z[5] = matthews_corrcoef(mm_train, model_xgb.predict(nn_train), sample_weight=sampleW)
                        z[6] = f1_score(Yy_Ext, model_xgb.predict(Xx_Ext), sample_weight=sampleExtW)
                        z[7] = matthews_corrcoef(Yy_Ext, model_xgb.predict(Xx_Ext), sample_weight=sampleExtW)
                        print("## No CV ##")
                        f1_10_train = cross_val_score(model_xgb, Xx_train, Yy_train, cv = 3, scoring=f1)
                        z[8] = avgScore(f1_10_train, 3)
                        #z[8] = 0
                        z[9] = f1_score(Yy_FNA, model_xgb.predict(Xx_FNA), sample_weight=sampleFNAW)
                        z[10] = matthews_corrcoef(Yy_FNA, model_xgb.predict(Xx_FNA), sample_weight=sampleFNAW)
                        z[11] = recall_score(Yy_FNA, model_xgb.predict(Xx_FNA))
                        z[12] = rs
                        z[13] = importances["importances_mean"][0]
                        z[14] = importances["importances_mean"][1]
                        z[15] = importances["importances_mean"][2]
                        z[16] = importances["importances_mean"][3]
                        z[17] = importances["importances_mean"][4]
                        z[18] = importances["importances_mean"][5]
                        z[19] = importances["importances_mean"][6]
                        z[20] = importances["importances_mean"][7]
                        z[21] = importances["importances_mean"][8]
                        z[22] = importances["importances_mean"][9]
                        z[23] = importances["importances_mean"][10]
                        z[24] = importances["importances_mean"][11]
                        z[25] = importances["importances_mean"][12]
                        z[26] = importances["importances_mean"][13]
                        z[27] = importances["importances_mean"][14]
                        z[28] = importances["importances_mean"][15]
                        z[29] = importances["importances_mean"][16]
                        z[30] = importances["importances_mean"][17]
                        z[31] = importances["importances_std"][0]
                        z[32] = importances["importances_std"][1]
                        z[33] = importances["importances_std"][2]
                        z[34] = importances["importances_std"][3]
                        z[35] = importances["importances_std"][4]
                        z[36] = importances["importances_std"][5]
                        z[37] = importances["importances_std"][6]
                        z[38] = importances["importances_std"][7]
                        z[39] = importances["importances_std"][8]
                        z[40] = importances["importances_std"][9]
                        z[41] = importances["importances_std"][10]
                        z[42] = importances["importances_std"][11]
                        z[43] = importances["importances_std"][12]
                        z[44] = importances["importances_std"][13]
                        z[45] = importances["importances_std"][14]
                        z[46] = importances["importances_std"][15]
                        z[47] = importances["importances_std"][16]
                        z[48] = importances["importances_std"][17]
                    else:
                        itrain = f1_score(mm_train, model_xgb.predict(nn_train), sample_weight=sampleW)
                        jtrain = matthews_corrcoef(mm_train, model_xgb.predict(nn_train), sample_weight=sampleW)
                        ival = f1_score(Yy_Ext, model_xgb.predict(Xx_Ext), sample_weight=sampleExtW)
                        jval = matthews_corrcoef(Yy_Ext, model_xgb.predict(Xx_Ext), sample_weight=sampleExtW)
                        print("## No CV ##")
                        f1_10_train = cross_val_score(model_xgb, Xx_train, Yy_train, cv = 3, scoring=f1)
                        traincvtrain = avgScore(f1_10_train, 3) 
                        f1s = f1_score(Yy_FNA, model_xgb.predict(Xx_FNA), sample_weight=sampleFNAW)
                        mccs = matthews_corrcoef(Yy_FNA, model_xgb.predict(Xx_FNA), sample_weight=sampleFNAW)
                        rec = recall_score(Yy_FNA, model_xgb.predict(Xx_FNA))
                        im1 = importances["importances_mean"][0]
                        im2 = importances["importances_mean"][1]
                        im3 = importances["importances_mean"][2]
                        im4 = importances["importances_mean"][3]
                        im5 = importances["importances_mean"][4]
                        im6 = importances["importances_mean"][5]
                        im7 = importances["importances_mean"][6]
                        im8 = importances["importances_mean"][7]
                        im9 = importances["importances_mean"][8]
                        im10 = importances["importances_mean"][9]
                        im11 = importances["importances_mean"][10]
                        im12 = importances["importances_mean"][11]
                        im13 = importances["importances_mean"][12]
                        im14 = importances["importances_mean"][13]
                        im15 = importances["importances_mean"][14]
                        im16 = importances["importances_mean"][15]
                        im17 = importances["importances_mean"][16]
                        im18 = importances["importances_mean"][17]
                        sd1 = importances["importances_std"][0]
                        sd2 = importances["importances_std"][1]
                        sd3 = importances["importances_std"][2]
                        sd4 = importances["importances_std"][3]
                        sd5 = importances["importances_std"][4]
                        sd6 = importances["importances_std"][5]
                        sd7 = importances["importances_std"][6]
                        sd8 = importances["importances_std"][7]
                        sd9 = importances["importances_std"][8]
                        sd10 = importances["importances_std"][9]
                        sd11 = importances["importances_std"][10]
                        sd12 = importances["importances_std"][11]
                        sd13 = importances["importances_std"][12]
                        sd14 = importances["importances_std"][13]
                        sd15 = importances["importances_std"][14]
                        sd16 = importances["importances_std"][15]
                        sd17 = importances["importances_std"][16]
                        sd18 = importances["importances_std"][17]
                        tem = np.array([lr, est, d, t, itrain, jtrain, ival, jval, traincvtrain, f1s, mccs, rec, rs, im1, im2, im3, im4, im5, im6, im7, im8, im9, im10, im11, im12, im13, im14, im15, im16, im17, im18, sd1, sd2, sd3, sd4, sd5, sd6, sd7, sd8, sd9, sd10, sd11, sd12, sd13, sd14, sd15, sd16, sd17, sd18])
                        z = np.row_stack([z, tem])
                        print(z[-1])
                    print("End of ", itr, " iterations")
                    itr += 1
    z_df = pd.DataFrame(z, columns=["lr", "est", "depth", "tree", 
                                    "f1_train", "mcc_train", "f1_ext", "mcc_ext", 
                                    "f1_3Ftrain", "f1_fna", "mcc_fna", "recall", "state", 
                                    "im1", "im2", "im3", "im4", "im5", "im6", "im7", "im8", "im9", 
                                    "im10", "im11", "im12", "im13", "im14", "im15", "im16", "im17", "im18", 
                                    "sd1", "sd2", "sd3", "sd4", "sd5", "sd6", "sd7", "sd8", "sd9", 
                                    "sd10", "sd11", "sd12", "sd13", "sd14", "sd15", "sd16", "sd17", "sd18"])

    return z_df


In [26]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
## call Gradient Boost ##
optiSearch_df = optimXGBoostClass(trainDEFSDf, ingestedDEFSDf, extDEFSDf, fnaDEFSDf)

itr= 1 , lr= 100 , est= 2 , depth= 5 , tree= 25
## loading in data ##
## Classification ##
[0]	validation_0-auc:0.95678	validation_1-auc:0.50113
[1]	validation_0-auc:0.85757	validation_1-auc:0.74063
[2]	validation_0-auc:0.94401	validation_1-auc:0.51815
[ 0.00201802  0.03708526  0.06375125  0.04071093  0.01025693  0.00120292
  0.          0.02367955 -0.0254766   0.00417695  0.00199434  0.
  0.00101915  0.06104328  0.          0.03468281  0.         -0.00538551]
## No CV ##
End of  1  iterations
itr= 2 , lr= 100 , est= 3 , depth= 5 , tree= 25
## loading in data ##
## Classification ##
[0]	validation_0-auc:0.95678	validation_1-auc:0.50113
[1]	validation_0-auc:0.85757	validation_1-auc:0.74063
[2]	validation_0-auc:0.94401	validation_1-auc:0.51815
[3]	validation_0-auc:0.93390	validation_1-auc:0.65856
[4]	validation_0-auc:0.95779	validation_1-auc:0.50982
[ 0.00201802  0.03708526  0.06375125  0.04071093  0.01025693  0.00120292
  0.          0.02367955 -0.0254766   0.00417695  0.00199434  0.
  

[1]	validation_0-auc:0.85757	validation_1-auc:0.74063
[2]	validation_0-auc:0.94401	validation_1-auc:0.51815
[3]	validation_0-auc:0.93390	validation_1-auc:0.65856
[4]	validation_0-auc:0.95779	validation_1-auc:0.50982
[ 0.00201802  0.03708526  0.06375125  0.04071093  0.01025693  0.00120292
  0.          0.02367955 -0.0254766   0.00417695  0.00199434  0.
  0.00101915  0.06104328  0.          0.03468281  0.         -0.00538551]
## No CV ##
[ 1.00000000e+02  3.00000000e+00  5.00000000e+00  5.00000000e+01
  8.62303697e-01  7.22274849e-01  8.24976118e-01  6.29363570e-01
             nan  7.82421589e-01  5.21238379e-01  9.32655511e-01
  4.20000000e+01  2.01801558e-03  3.70852640e-02  6.37512542e-02
  4.07109277e-02  1.02569306e-02  1.20291767e-03  0.00000000e+00
  2.36795527e-02 -2.54766012e-02  4.17695404e-03  1.99434054e-03
  0.00000000e+00  1.01915424e-03  6.10432802e-02  0.00000000e+00
  3.46828108e-02  0.00000000e+00 -5.38550862e-03  1.33489003e-04
  7.41561828e-04  8.51943578e-04  4.3705

[1]	validation_0-auc:0.85757	validation_1-auc:0.74063
[2]	validation_0-auc:0.94401	validation_1-auc:0.51815
[3]	validation_0-auc:0.93390	validation_1-auc:0.65856
[4]	validation_0-auc:0.95779	validation_1-auc:0.50982
[5]	validation_0-auc:0.94132	validation_1-auc:0.60771
[ 0.00201802  0.03708526  0.06375125  0.04071093  0.01025693  0.00120292
  0.          0.02367955 -0.0254766   0.00417695  0.00199434  0.
  0.00101915  0.06104328  0.          0.03468281  0.         -0.00538551]
## No CV ##
[ 1.00000000e+02  5.00000000e+00  5.00000000e+00  1.00000000e+02
  8.62303697e-01  7.22274849e-01  8.24976118e-01  6.29363570e-01
             nan  7.82421589e-01  5.21238379e-01  9.32655511e-01
  4.20000000e+01  2.01801558e-03  3.70852640e-02  6.37512542e-02
  4.07109277e-02  1.02569306e-02  1.20291767e-03  0.00000000e+00
  2.36795527e-02 -2.54766012e-02  4.17695404e-03  1.99434054e-03
  0.00000000e+00  1.01915424e-03  6.10432802e-02  0.00000000e+00
  3.46828108e-02  0.00000000e+00 -5.38550862e-03  1

[ 1.00000000e+02  5.00000000e+00  6.00000000e+00  2.50000000e+01
  9.76542706e-01  9.52817228e-01  8.18141975e-01  6.09868226e-01
             nan  6.61565446e-01  2.97182051e-01  6.87824098e-01
  4.20000000e+01  1.19502599e-04 -9.79695832e-03  6.96576138e-02
  3.38688403e-02  2.04721480e-02  6.89958400e-04  4.09014555e-03
  3.16253481e-02 -1.35567806e-02  5.95145489e-03  8.81613511e-04
  4.84774693e-05 -7.89168104e-06 -2.97189434e-02 -5.84018219e-02
  1.70494132e-02 -3.34945491e-03  9.48467323e-03  7.76504227e-05
  3.54184915e-04  6.16278535e-04  1.26331101e-03  3.76196408e-04
  6.66587547e-05  1.56523641e-04  6.18414539e-04  1.89067878e-04
  2.17526499e-04  1.33945247e-04  7.05943437e-05  7.21877345e-06
  4.41711194e-04  8.85351755e-04  4.51180031e-04  7.73634501e-05
  1.00328283e-03]
End of  18  iterations
itr= 19 , lr= 100 , est= 7 , depth= 6 , tree= 25
## loading in data ##
## Classification ##
[0]	validation_0-auc:0.96932	validation_1-auc:0.62389
[1]	validation_0-auc:0.96529	vali

[1]	validation_0-auc:0.96529	validation_1-auc:0.64812
[2]	validation_0-auc:0.96198	validation_1-auc:0.62291
[3]	validation_0-auc:0.97487	validation_1-auc:0.58982
[4]	validation_0-auc:0.97483	validation_1-auc:0.58465
[5]	validation_0-auc:0.97796	validation_1-auc:0.64212
[6]	validation_0-auc:0.98426	validation_1-auc:0.62534
[7]	validation_0-auc:0.98059	validation_1-auc:0.57224
[8]	validation_0-auc:0.98480	validation_1-auc:0.54113
[ 1.19502599e-04 -9.79695832e-03  6.96576138e-02  3.38688403e-02
  2.04721480e-02  6.89958400e-04  4.09014555e-03  3.16253481e-02
 -1.35567806e-02  5.95145489e-03  8.81613511e-04  4.84774693e-05
 -7.89168104e-06 -2.97189434e-02 -5.84018219e-02  1.70494132e-02
 -3.34945491e-03  9.48467323e-03]
## No CV ##
[ 1.00000000e+02  7.00000000e+00  6.00000000e+00  5.00000000e+01
  9.76542706e-01  9.52817228e-01  8.18141975e-01  6.09868226e-01
             nan  6.61565446e-01  2.97182051e-01  6.87824098e-01
  4.20000000e+01  1.19502599e-04 -9.79695832e-03  6.96576138e-02
  

[6]	validation_0-auc:0.98426	validation_1-auc:0.62534
[7]	validation_0-auc:0.98059	validation_1-auc:0.57224
[ 1.19502599e-04 -9.79695832e-03  6.96576138e-02  3.38688403e-02
  2.04721480e-02  6.89958400e-04  4.09014555e-03  3.16253481e-02
 -1.35567806e-02  5.95145489e-03  8.81613511e-04  4.84774693e-05
 -7.89168104e-06 -2.97189434e-02 -5.84018219e-02  1.70494132e-02
 -3.34945491e-03  9.48467323e-03]
## No CV ##
[ 1.00000000e+02  7.00000000e+00  6.00000000e+00  1.00000000e+02
  9.76542706e-01  9.52817228e-01  8.18141975e-01  6.09868226e-01
             nan  6.61565446e-01  2.97182051e-01  6.87824098e-01
  4.20000000e+01  1.19502599e-04 -9.79695832e-03  6.96576138e-02
  3.38688403e-02  2.04721480e-02  6.89958400e-04  4.09014555e-03
  3.16253481e-02 -1.35567806e-02  5.95145489e-03  8.81613511e-04
  4.84774693e-05 -7.89168104e-06 -2.97189434e-02 -5.84018219e-02
  1.70494132e-02 -3.34945491e-03  9.48467323e-03  7.76504227e-05
  3.54184915e-04  6.16278535e-04  1.26331101e-03  3.76196408e-04
 

[3]	validation_0-auc:0.97058	validation_1-auc:0.70092
[4]	validation_0-auc:0.97387	validation_1-auc:0.72915
[5]	validation_0-auc:0.96828	validation_1-auc:0.68233
[6]	validation_0-auc:0.97855	validation_1-auc:0.69837
[7]	validation_0-auc:0.97390	validation_1-auc:0.67384
[8]	validation_0-auc:0.98011	validation_1-auc:0.71221
[9]	validation_0-auc:0.97421	validation_1-auc:0.72002
[10]	validation_0-auc:0.98458	validation_1-auc:0.71733
[11]	validation_0-auc:0.97492	validation_1-auc:0.74077
[12]	validation_0-auc:0.98560	validation_1-auc:0.76135
[13]	validation_0-auc:0.97673	validation_1-auc:0.71214
[14]	validation_0-auc:0.98343	validation_1-auc:0.68545
[15]	validation_0-auc:0.97937	validation_1-auc:0.63946
[16]	validation_0-auc:0.98623	validation_1-auc:0.64143
[17]	validation_0-auc:0.98063	validation_1-auc:0.62981
[18]	validation_0-auc:0.98591	validation_1-auc:0.64399
[ 0.00114542  0.00435283  0.07245352  0.01274506  0.05406816  0.01294461
  0.06650094  0.02804703  0.01954995  0.00822088 -0.00

KeyboardInterrupt: 

In [ ]:
## save ##
optiSearch_df.to_csv(r"H:\3_output_raMSIn\hyperparameterTuning_modelSelection_XGB4.csv")

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [28]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [32]:
df_ROI_for_ML_Opti = pd.concat([trainDEFSDf, extDEFSDf, fnaDEFSDf]).set_index('pixel_id')

In [41]:
df_ROI_for_ML_Opti_ingested = pd.concat([trainDEFSDf, extDEFSDf]).set_index('pixel_id')

In [44]:
df_ROI_for_ML_Opti_train = trainDEFSDf.set_index('pixel_id')

In [47]:
df_ROI_for_ML_Opti_ext = extDEFSDf.set_index('pixel_id')

In [50]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

## 2.2. PCA Analysis

In [51]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_FNA):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_FNA.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/PCA_fna.tif", bbox_inches = 'tight')

    """components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()"""

In [52]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [138]:
def split_dataset(df=df_ROI_for_ML_Opti):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2, random_state=42)
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti)

In [139]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_train.columns[:-1]]

In [140]:
y_ingested = df_ROI_for_ML_Opti_ingested["type"]

In [141]:
X_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[:-1]]

In [142]:
y_train = df_ROI_for_ML_Opti_train["type"]

In [143]:
X_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[:-1]]

In [144]:
y_val = df_ROI_for_ML_Opti_ext["type"]

In [145]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [146]:
y_FNA = df_ROI_for_ML_Opti_FNA["type"]

## 3.2 Machine Learning Modeling: Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [147]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### 3.2.1. Logistic Regression

In [166]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.0005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_10_train = cross_val_score(model_lr, XTrain, yTrain, cv = 3, scoring=f1)
    print("3F-CV:", avgScore(f1_10_train, 3))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    feature_importances = model_lr.coef_[0]
    feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    fig = px.bar(feature_df, x='coef', y='m/z')
    fig.show()
    
    return x, y

In [167]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.883231466964114
Training Set MCC: 0.758363117857726
Training Set Accuracy: 0.874769129287599
              precision    recall  f1-score   support

           0       0.93      0.82      0.87     47449
           1       0.83      0.93      0.88     43511

    accuracy                           0.87     90960
   macro avg       0.88      0.88      0.87     90960
weighted avg       0.88      0.87      0.87     90960

[[39108  8341]
 [ 3050 40461]]
Ext Val Set F1: 0.8320024636840297
Ext Val Set MCC: 0.6435701484536753
Ext Val Set Accuracy: 0.8186008230452675
              precision    recall  f1-score   support

           0       0.89      0.72      0.79      2943
           1       0.77      0.91      0.84      3132

    accuracy                           0.82      6075
   macro avg       0.83      0.82      0.82      6075
weighted avg       0.83      0.82      0.82      6075

[[2112  831]
 [ 271 2861]]
3F-CV: 0.6688314685572054
FNA Set F1: 0.8094300016856857
FNA Set

### 3.2.2. XBG

In [170]:
## Build XGBoost Model
import xgboost as xgb


def XGBoost_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):

    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 1, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 6, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 0, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 1, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 10)

    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XTrain, yTrain), (XTest, yTest)])
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_xgb.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_xgb.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_xgb.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_xgb.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_xgb.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_xgb.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:",metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_10_train = cross_val_score(model_xgb, XTrain, yTrain, cv = 3, scoring=f1)
    print("3F-CV:", avgScore(f1_10_train, 3))
    
    # Test set
    predictions_FNA = model_xgb.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_xgb.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_xgb.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_xgb.predict(XTest)))
    print("FNA Test Set Accuracy:",metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_xgb.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti,x = feature_imp[:20].index,color= 'type')
    #fig.show()
    
    return x, y

In [171]:
coeff_XGB, TOPfeatures_XGB = XGBoost_func()

[0]	validation_0-auc:0.96932	validation_1-auc:0.62389
[1]	validation_0-auc:0.96527	validation_1-auc:0.63918
[2]	validation_0-auc:0.97209	validation_1-auc:0.75335
[3]	validation_0-auc:0.97017	validation_1-auc:0.74248
[4]	validation_0-auc:0.98167	validation_1-auc:0.66606
[5]	validation_0-auc:0.98271	validation_1-auc:0.75312
[6]	validation_0-auc:0.98503	validation_1-auc:0.64401
[7]	validation_0-auc:0.98514	validation_1-auc:0.72395
[8]	validation_0-auc:0.98664	validation_1-auc:0.63513
[9]	validation_0-auc:0.98607	validation_1-auc:0.72804
[10]	validation_0-auc:0.98786	validation_1-auc:0.66241
[11]	validation_0-auc:0.98655	validation_1-auc:0.76371
[12]	validation_0-auc:0.98708	validation_1-auc:0.79337
[13]	validation_0-auc:0.98809	validation_1-auc:0.78157
[14]	validation_0-auc:0.98713	validation_1-auc:0.75580
[15]	validation_0-auc:0.98722	validation_1-auc:0.69760
[16]	validation_0-auc:0.98736	validation_1-auc:0.68589
[17]	validation_0-auc:0.98790	validation_1-auc:0.67375
[18]	validation_0-au

## 3.3 Machine Learning Modeling: Model 1 (Train 6:6, Val 4:4, Test 3:3)

### 3.3.1. Logistic Regression

In [185]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.0005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(XTrain)
    print("Training Set F1:", f1_score(yTrain, model_lr.predict(XTrain), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yTrain, model_lr.predict(XTrain), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yTrain, X_train_predictions))
    print(classification_report(yTrain, X_train_predictions))
    print(confusion_matrix(yTrain, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yTrain, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_10_train = cross_val_score(model_lr, XTrain, yTrain, cv = 3, scoring=f1)
    print("3F-CV:", avgScore(f1_10_train, 3))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    feature_importances = model_lr.coef_[0]
    feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    fig = px.bar(feature_df, x='coef', y='m/z')
    fig.show()
    
    return x, y

In [186]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.8842491991260675
Training Set MCC: 0.7603949218110979
Training Set Accuracy: 0.8755496921723834
              precision    recall  f1-score   support

           0       0.93      0.82      0.87     47449
           1       0.83      0.93      0.88     43511

    accuracy                           0.88     90960
   macro avg       0.88      0.88      0.88     90960
weighted avg       0.88      0.88      0.88     90960

[[39062  8387]
 [ 2933 40578]]
Ext Val Set F1: 0.8091641942302281
Ext Val Set MCC: 0.5926357166725181
Ext Val Set Accuracy: 0.7937448559670782
              precision    recall  f1-score   support

           0       0.85      0.69      0.77      2943
           1       0.76      0.89      0.82      3132

    accuracy                           0.79      6075
   macro avg       0.80      0.79      0.79      6075
weighted avg       0.80      0.79      0.79      6075

[[2043  900]
 [ 353 2779]]
3F-CV: 0.6548205699454676
FNA Set F1: 0.7809312470954997
FNA 

In [187]:
## Build XGBoost Model
import xgboost as xgb


def XGBoost_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):

    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 1, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 6, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 0, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 1, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 10)

    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XTrain, yTrain), (XTest, yTest)])
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_xgb.predict(XTrain)
    print("Training Set F1:", f1_score(yTrain, model_xgb.predict(XTrain), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yTrain, model_xgb.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yTrain, X_train_predictions))
    print(classification_report(yTrain, X_train_predictions))
    print(confusion_matrix(yTrain, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yTrain, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_xgb.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_xgb.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_xgb.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:",metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_10_train = cross_val_score(model_xgb, XTrain, yTrain, cv = 3, scoring=f1)
    print("3F-CV:", avgScore(f1_10_train, 3))
    
    # Test set
    predictions_FNA = model_xgb.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_xgb.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_xgb.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_xgb.predict(XTest)))
    print("FNA Test Set Accuracy:",metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_xgb.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti,x = feature_imp[:20].index,color= 'type')
    #fig.show()
    
    return x, y

In [188]:
coeff_XGB, TOPfeatures_XGB = XGBoost_func()

[0]	validation_0-auc:0.97433	validation_1-auc:0.52106
[1]	validation_0-auc:0.94550	validation_1-auc:0.57293
[2]	validation_0-auc:0.97193	validation_1-auc:0.45906
[3]	validation_0-auc:0.96485	validation_1-auc:0.56072
[4]	validation_0-auc:0.97913	validation_1-auc:0.47383
[5]	validation_0-auc:0.97581	validation_1-auc:0.58594
[6]	validation_0-auc:0.98196	validation_1-auc:0.50860
[7]	validation_0-auc:0.98232	validation_1-auc:0.58844
[8]	validation_0-auc:0.98371	validation_1-auc:0.51337
[9]	validation_0-auc:0.98334	validation_1-auc:0.56537
[10]	validation_0-auc:0.98450	validation_1-auc:0.49807
[11]	validation_0-auc:0.98360	validation_1-auc:0.53236
[12]	validation_0-auc:0.98475	validation_1-auc:0.48785
[13]	validation_0-auc:0.98375	validation_1-auc:0.53516
[14]	validation_0-auc:0.98653	validation_1-auc:0.47759
[15]	validation_0-auc:0.98397	validation_1-auc:0.53477
[16]	validation_0-auc:0.98677	validation_1-auc:0.46869
Training Set F1: 0.9825136247859996
Training Set MCC: 0.9648746379975889
Tr

## 3.4 Machine Learning Modeling: Model 2 (Train 4:4, Val 6:6, Test 3:3)

In [193]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.0005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_lr.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_lr.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:",metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_10_train = cross_val_score(model_lr, XTrain, yTrain, cv = 3, scoring=f1)
    print("3F-CV:", avgScore(f1_10_train, 3))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:",metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    feature_importances = model_lr.coef_[0]
    feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    fig = px.bar(feature_df, x='coef', y='m/z')
    fig.show()
    
    return x, y

In [194]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.0
Training Set MCC: 0.0
Training Set Accuracy: 0.5216468777484609
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     47449
           1       0.00      0.00      0.00     43511

    accuracy                           0.52     90960
   macro avg       0.26      0.50      0.34     90960
weighted avg       0.27      0.52      0.36     90960

[[47449     0]
 [43511     0]]
Ext Val Set F1: 0.0
Ext Val Set MCC: 0.0
Ext Val Set Accuracy: 0.48444444444444446
              precision    recall  f1-score   support

           0       0.48      1.00      0.65      2943
           1       0.00      0.00      0.00      3132

    accuracy                           0.48      6075
   macro avg       0.24      0.50      0.33      6075
weighted avg       0.23      0.48      0.32      6075

[[2943    0]
 [3132    0]]
3F-CV: 0.0
FNA Set F1: 0.0
FNA Set MCC: 0.0
FNA Set Recall: 0.0
FNA Test Set Accuracy: 0.5021363907960451
              

In [189]:
## Build XGBoost Model
import xgboost as xgb


def XGBoost_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):

    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 1, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 6, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 0, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 1, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 10)

    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XTrain, yTrain), (XTest, yTest)])
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_xgb.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_xgb.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_xgb.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_xgb.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_xgb.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_xgb.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:",metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_10_train = cross_val_score(model_xgb, XTrain, yTrain, cv = 3, scoring=f1)
    print("3F-CV:", avgScore(f1_10_train, 3))
    
    # Test set
    predictions_FNA = model_xgb.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_xgb.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_xgb.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_xgb.predict(XTest)))
    print("FNA Test Set Accuracy:",metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_xgb.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti,x = feature_imp[:20].index,color= 'type')
    #fig.show()
    
    return x, y

In [190]:
coeff_XGB, TOPfeatures_XGB = XGBoost_func()

[0]	validation_0-auc:0.97143	validation_1-auc:0.56335
[1]	validation_0-auc:0.83031	validation_1-auc:0.56474
[2]	validation_0-auc:0.77260	validation_1-auc:0.49493
[3]	validation_0-auc:0.90628	validation_1-auc:0.51820
[4]	validation_0-auc:0.76489	validation_1-auc:0.49970
[5]	validation_0-auc:0.91489	validation_1-auc:0.51850
[6]	validation_0-auc:0.76642	validation_1-auc:0.49319
[7]	validation_0-auc:0.90478	validation_1-auc:0.53482
[8]	validation_0-auc:0.76992	validation_1-auc:0.49298
[9]	validation_0-auc:0.90289	validation_1-auc:0.53376
[10]	validation_0-auc:0.76992	validation_1-auc:0.49298
[11]	validation_0-auc:0.89743	validation_1-auc:0.53376
Training Set F1: 0.6396475936790568
Training Set MCC: 0.39309105505436515
Training Set Accuracy: 0.6949538258575197
              precision    recall  f1-score   support

           0       0.67      0.83      0.74     47449
           1       0.74      0.55      0.63     43511

    accuracy                           0.69     90960
   macro avg    